# face expresion using model. it workng in fine laptop and pc both.

In [ ]:
import os
import io
import numpy as np
import cv2
from PIL import Image
from flask import Flask, request, jsonify, render_template, redirect, url_for
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import tempfile

#face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+ 'haarcascade_frontalface_default.xml')
# Load the saved model
model = load_model('model.h5')
classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Initialize the Flask app
app = Flask(__name__)

# Set the upload folder
# Configure the temporary upload folder
app.config['UPLOAD_FOLDER'] = 'C:/Users/RAJESH KUMAR/Desktop/working/working2/temporyry'


# Define a function to preprocess the image
def preprocess_image(image):
    # Convert the image to grayscale
    image = image.convert('L')
    # Resize the image to 48x48 pixels
    image = image.resize((48, 48))
    # Convert the image to a NumPy array
    image = img_to_array(image)
    # Reshape the array to (1, 48, 48, 1)
    image = np.reshape(image, (1, 48, 48, 1))
    # Normalize the image pixels to the range [0, 1]
    image = image / 255.0
    # Return the preprocessed image
    return image

# Define the route for the home page
@app.route('/')
def home():
    # Render the HTML template
    return render_template('index.html')

# Define the route for the prediction API
@app.route('/predict', methods=['POST'])
def predict():
    # Get the video file from the request
    file = request.files['video']
    
    # Save the file to a temporary folder
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(file_path)
    
    # Read the video file as an OpenCV VideoCapture object
    video = cv2.VideoCapture(file_path)
    
    # Create a window to display the video frames
    cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Video', 640, 480)
    
    # Loop through the frames in the video
    while True:
        # Read the next frame from the video
        ret, frame = video.read()
        
        # If there are no more frames, break out of the loop
        if not ret:
            break
        
        # Convert the frame to a PIL image
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        # Preprocess the image
        image = preprocess_image(image)
        
        # Make a prediction using the model
        prediction = model.predict(image)
        
        # Get the index of the predicted class
        predicted_class = np.argmax(prediction)
        
        # Get the predicted emotion label
        emotion = classes[predicted_class]
        
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Draw the predicted emotion label on the frame
        cv2.putText(frame, emotion, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        
        # Display the frame
        cv2.imshow('Video', frame)
        
        # Wait for a key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the video object and close the window
    video.release()
    cv2.destroyAllWindows()

    # Redirect to the result page
    return redirect(url_for('result', emotion=emotion))

# Define the route for the result page
@app.route('/result/<emotion>')
def result(emotion):
    # Render the result page
    return render_template('result.html', emotion=emotion)


            # Start the Flask app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=9090)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.42.217:9090/ (Press CTRL+C to quit)
192.168.42.217 - - [27/Mar/2023 13:20:25] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 34ms/step


192.168.42.217 - - [27/Mar/2023 13:20:35] "POST /predict HTTP/1.1" 302 -
192.168.42.217 - - [27/Mar/2023 13:20:35] "GET /result/Neutral HTTP/1.1" 200 -


# face expredion from video using deep face laptop working.

In [ ]:
#Deepface way

import os
import io
import numpy as np
import cv2
from PIL import Image
from flask import Flask, request, jsonify, render_template, redirect, url_for
from deepface import DeepFace

#face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+ 'haarcascade_frontalface_default.xml')

# Initialize the Flask app
app = Flask(__name__)


# Configure the temporary upload folder
app.config['UPLOAD_FOLDER'] = 'C:/Users/RAJESH KUMAR/Desktop/working/working2/temporyry'


# Define the route for the home page
@app.route('/')
def home():
    # Render the HTML template
    return render_template('index.html')

# Define the route for the prediction API
@app.route('/predict', methods=['POST'])
def predict():
    # Get the video file from the request
    file = request.files['video']
    # Save the file to a temporary location on the server
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(file_path)
    # Read the video file as an OpenCV VideoCapture object
    video = cv2.VideoCapture(file_path)
    # Loop through the frames in the video
    while True:
        # Read the next frame from the video
        ret, frame = video.read()
        # If there are no more frames, break out of the loop
        if not ret:
            break
        # Make a prediction using DeepFace library
        prediction = DeepFace.analyze(frame, enforce_detection=False, actions=['emotion'])
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Get the predicted emotion label
        predicted_emotion = prediction['dominant_emotion']
        print(predicted_emotion)
        # Display the predicted emotion on the frame
        cv2.putText(frame,
            prediction['dominant_emotion'],
            (50,50),
            cv2.FONT_HERSHEY_SIMPLEX,3,
            (255, 0, 0),
            2,cv2.LINE_4);
        #cv2.putText(frame, predicted_emotion, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        # Display the frame
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # Release the video capture object
    video.release()
    # Destroy all OpenCV windows
    cv2.destroyAllWindows()
    # Redirect to the home page
    return redirect(url_for('home'))
            # Start the Flask app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=9090)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.42.217:9090/ (Press CTRL+C to quit)
192.168.42.217 - - [27/Mar/2023 13:12:33] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 223ms/step
sad
1/1 [==============================] - 0s 33ms/step
sad
1/1 [==============================] - 0s 29ms/step
angry
1/1 [==============================] - 0s 31ms/step
fear
1/1 [==============================] - 0s 33ms/step
fear
1/1 [==============================] - 0s 29ms/step
angry
1/1 [==============================] - 0s 32ms/step
angry
1/1 [==============================] - 0s 31ms/step
angry
1/1 [==============================] - 0s 33ms/step
angry
1/1 [==============================] - 0s 35ms/step
angry
1/1 [==============================] - 0s 37ms/step
angry
1/1 [==============================] - 0s 29ms/step
angry
1/1 [==============================] - 0s 35ms/step
angry
1/1 [==============================] - 0s 30ms/step
angry
1/1 [==============================] - 0s 33ms/step
angry
1/1 [==============================] - 0s 28ms/step
angry
1/1 [==============================] - 0s 27ms/step
sad
1/1 [================

192.168.42.217 - - [27/Mar/2023 13:12:50] "POST /predict HTTP/1.1" 302 -
192.168.42.217 - - [27/Mar/2023 13:12:50] "GET / HTTP/1.1" 200 -


angry


# if you are using deepfaec updated like 0.78 or higher the u can use this code.

In [ ]:
#deepface way


import os
import io
import numpy as np
import cv2
from PIL import Image
from flask import Flask, request, jsonify, render_template, redirect, url_for
from deepface import DeepFace

#face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+ 'haarcascade_frontalface_default.xml')

# Initialize the Flask app
app = Flask(__name__)

# Configure the temporary upload folder
app.config['UPLOAD_FOLDER'] = 'C:/Users/Rajesh Kumar/Desktop/abcd/video flaskapp/temprory'


# Define the route for the home page
@app.route('/')
def home():
    # Render the HTML template
    return render_template('index.html')

# Define the route for the prediction API
@app.route('/predict', methods=['POST'])
def predict():
    # Get the video file from the request
    file = request.files['video']
    # Save the file to a temporary location on the server
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(file_path)
    # Read the video file as an OpenCV VideoCapture object
    video = cv2.VideoCapture(file_path)
    # Loop through the frames in the video
    while True:
        # Read the next frame from the video
        ret, frame = video.read()
        # If there are no more frames, break out of the loop
        if not ret:
            break
        # Make a prediction using DeepFace library
        prediction = DeepFace.analyze(frame, enforce_detection=False, actions=['emotion'])
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Get the predicted emotion label
        predicted_emotion = prediction[0]['dominant_emotion']
        print(predicted_emotion)
        # Display the predicted emotion on the frame
        cv2.putText(frame,
            prediction[0]['dominant_emotion'],
            (50,50),
            cv2.FONT_HERSHEY_SIMPLEX,3,
            (255, 0, 0),
            2,cv2.LINE_4);
        #cv2.putText(frame, predicted_emotion, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        # Display the frame
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # Release the video capture object
    video.release()
    # Destroy all OpenCV windows
    cv2.destroyAllWindows()
    # Redirect to the home page
    return redirect(url_for('home'))
            # Start the Flask app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=9090)